In [42]:
import pandas as pd
import re
import plotly.graph_objects as go
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import networkx as nx
import random
import plotly.express as px
from datetime import datetime

In [43]:
data = pd.read_csv('women_100m.csv')

In [44]:

data = pd.read_csv('women_100m.csv')
def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)
# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.replace(',', '.').astype(float)

# Dodanie nowej kolumny 'Year' na podstawie kolumny 'Event Date'
data['Year'] = data['Event Date'].dt.year

# Filtrowanie danych od roku 1975
data = data[data['Year'] >= 1975]

# Grupowanie danych według roku i wybór najlepszego wyniku dla każdego roku
best_results = data.groupby('Year')['Time'].min()

# Tworzenie wykresu za pomocą Plotly
fig = go.Figure()
data_help = data.copy()
best_results_names = data_help.groupby('Year').apply(lambda x: x.loc[x['Time'].idxmin(), ['Time', 'Athlete']])
top_runner_names_each_year = best_results_names['Athlete'].unique().tolist()
top_runner_names_each_year
color_names = [
    "AliceBlue", "Green", "Aqua", "Aquamarine", "Gold", "Beige",
    "Bisque", "Black", "BlanchedAlmond", "Blue", "BlueViolet", "Brown",
    "BurlyWood", "CadetBlue", "Chartreuse", "Chocolate", "Coral", "CornflowerBlue",
    "Cornsilk", "Crimson", "Cyan", "DarkBlue", "DarkCyan", "DarkGoldenRod",
    "DarkGray", "DarkGreen", "DarkKhaki", "DarkMagenta", "DarkOliveGreen",
    "DarkOrange", "DarkOrchid", "DarkRed", "DarkSalmon", "DarkSeaGreen",
    "DarkSlateBlue", "DarkSlateGray", "DarkTurquoise", "DarkViolet", "DeepPink",
    "DeepSkyBlue", "DimGray", "DodgerBlue", "FireBrick", "FloralWhite",
    "ForestGreen", "Fuchsia", "Gainsboro", "GhostWhite", "Gold", "GoldenRod",
    "Gray", "Green", "GreenYellow", "HoneyDew", "HotPink", "IndianRed", "Indigo",
    "Ivory", "Khaki", "Lavender", "LavenderBlush", "LawnGreen", "LemonChiffon",
    "LightBlue", "LightCoral", "LightCyan", "LightGoldenRodYellow", "LightGray",
    "LightGreen", "LightPink", "LightSalmon", "LightSeaGreen", "LightSkyBlue",
    "LightSlateGray", "LightSteelBlue", "LightYellow", "Lime", "LimeGreen",
    "Linen", "Magenta", "Maroon", "MediumAquaMarine", "MediumBlue", "MediumOrchid",
    "MediumPurple", "MediumSeaGreen", "MediumSlateBlue", "MediumSpringGreen",
    "MediumTurquoise", "MediumVioletRed", "MidnightBlue", "MintCream", "MistyRose",
    "Moccasin", "NavajoWhite", "Navy", "OldLace", "Olive", "OliveDrab",
    "Orange", "OrangeRed", "Orchid", "PaleGoldenRod", "PaleGreen", "PaleTurquoise",
    "PaleVioletRed", "PapayaWhip", "PeachPuff", "Peru", "Pink", "Plum",
    "PowderBlue", "Purple", "Red", "RosyBrown", "RoyalBlue", "SaddleBrown",
    "Salmon", "SandyBrown", "SeaGreen", "SeaShell", "Sienna", "Silver",
    "SkyBlue", "SlateBlue", "SlateGray", "Snow", "SpringGreen", "SteelBlue",
    "Tan", "Teal", "Thistle", "Tomato", "Turquoise", "Violet", "Wheat", "White",
    "WhiteSmoke", "Yellow", "YellowGreen"
]

colors = dict(zip(top_runner_names_each_year, color_names[:len(top_runner_names_each_year)]))
for year, time in best_results.items():
    athlete_data = data.loc[(data['Year'] == year) & (data['Time'] == time)]
    athlete = athlete_data['Athlete'].values[0]
    country = athlete_data['Country'].values[0]
    color = colors.get(athlete, 'gray')
    label = f"Zawodniczka: {athlete}<br>Rok: {int(year)}<br>Czas: {time} s<br>Kraj: {country}"
    fig.add_trace(go.Scatter(
        x=[year],
        y=[time],
        mode='markers',
        name=athlete,
        text=[label],
        hovertemplate='%{text}<extra></extra>',
        marker=dict(color=color, size=15),  # Zwiększenie rozmiaru kulek
    ))


# Konfiguracja układu wykresu
fig.update_layout(
    xaxis=dict(title='Rok'),
    yaxis=dict(title='Najlepszy czas (s)'),
    title='Najlepszy wynik w biegu na 100 m',
    showlegend=True,
    height= 700
)

# Ustawienie skalowania osi Y
fig.update_yaxes(range=[10.2, 11.3])

# Zapisywanie wykresu do pliku HTML
fig.write_html('best_of_every_season.html')

In [45]:
# tu bierzemy brzydko 11 i wyrzucamy jedno bo dwukrotnie mamy niemcy -> jako przed obaleniem ustroju i po
top_10_countries = data['Country'].value_counts().head(11)
country_codes = top_10_countries.index.tolist()
occurrences = top_10_countries.tolist()

country_to_remove = 'GDR'
if country_to_remove in country_codes:
    index_to_remove = country_codes.index(country_to_remove)
    country_codes.remove(country_to_remove)
    germany_index = country_codes.index('GER')
    occurrences[germany_index] += occurrences[index_to_remove]
    occurrences.pop(index_to_remove)

print("Country Codes:", country_codes)
print("Occurrences:", occurrences)

Country Codes: ['USA', 'JAM', 'BAH', 'CIV', 'NGR', 'TTO', 'GER', 'GBR', 'RUS', 'FRA']
Occurrences: [1053, 636, 122, 118, 118, 86, 82, 73, 69, 68]


In [46]:
#  this is the list that has to be customized for each file separetely due
#to the images being sourced from github folder

pictures = ['US.png', 'JM.png', 'BH.png', 'IE.png' , 'NG.png', 'TT.png', 'DE.png' ,'GB.png', 'RU.png', 'FR.png']

In [47]:
countires_flags = {
    'Country' : country_codes,
    'Count' : occurrences,
    'Picture' : pictures
}

In [48]:
# Wczytanie danych do DataFrame
df = pd.DataFrame(countires_flags)

# Tworzenie figury
fig = go.Figure()

# Tworzenie grafu
G = nx.Graph()

# Dodawanie wierzchołków do grafu
for i, row in df.iterrows():
    G.add_node(i)

# Generowanie losowego rozkładu kulek
pos = nx.spring_layout(G, iterations=2000, scale=0.5, seed=42)

# Dodawanie markerów z obrazkami i etykietami
for i, row in df.iterrows():
    x, y = pos[i]

    fig.add_trace(go.Scatter(
        x=[x],
        y=[y],
        mode='markers',
        marker=dict(
            size=30 * row['Count'] * 0.01,
            opacity=0.7,
            symbol='circle',
            color='rgba(0, 0, 0, 0)',
        ),
        hovertext=[f'Country: {row["Country"]}, Count: {row["Count"]}'],
        hoverinfo='text',
    ))
    fig.add_layout_image(
        dict(
            source=f"https://raw.githubusercontent.com/matahombres/CSS-Country-Flags-Rounded/master/flags/{row['Picture']}",
            xref="x",
            yref="y",
            x=x,
            y=y,
            sizex=0.001 * row['Count'],  # Mnożenie wartości Count przez 0.1
            sizey=0.001 * row['Count'],  # Mnożenie wartości Count przez 0.1
            xanchor="center",
            yanchor="middle",
            opacity=1,
            layer="above"
        )
    )

# Aktualizowanie osi x
fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False)

# Aktualizowanie osi y
fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False)

# Aktualizowanie układu wykresu
fig.update_layout(
    width=1150,
    height=600,
    plot_bgcolor="white",
    showlegend=False
)

# Zapisywanie wykresu do pliku HTML
fig.write_html("country_bubbles.html")


In [49]:

def calculate_age(birth_date, event_date):
    birth_date = datetime.strptime(birth_date, "%d.%m.%y")
    event_date = datetime.strptime(event_date, "%d.%m.%Y")
    
    birth_year = birth_date.year
    event_year = event_date.year
    
    if birth_year > 2010:
        birth_year = birth_year - 100
    
    age = event_year - birth_year
    if event_date.month < birth_date.month or (event_date.month == birth_date.month and event_date.day < birth_date.day):
        age -= 1
    return age


def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)


# Wczytanie danych z pliku CSV
data = pd.read_csv('women_100m.csv')
df = data.copy()

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.replace(',', '.').astype(float)

# Znalezienie najlepszego czasu dla każdego zawodnika
best_times = data.groupby('Athlete')['Time'].min()

all_ages = []
# Iteracja po zawodnikach i ich najlepszych czasach
for athlete, best_time in best_times.items():
    athlete_data = data[(data['Athlete'] == athlete) & (data['Time'] == best_time)]
    
    birth_date = athlete_data['Date of Birth'].values[0]
    event_date = athlete_data['Event Date'].values[0]

    # Sprawdzenie, czy birth_date lub event_date jest puste lub nie jest prawidłową datą
    if pd.isnull(birth_date) or pd.isnull(event_date) or not isinstance(birth_date, str) or not isinstance(event_date, str):
        continue
    
    try:
        age = calculate_age(birth_date, event_date)
        all_ages.append(age)
    except ValueError:
        continue

unique_values, frequencies = np.unique(all_ages, return_counts=True)
value_to_delete = 83

# Znalezienie indeksu wartości do usunięcia
index_to_delete = np.where(unique_values == value_to_delete)[0]
unique_values = np.delete(unique_values, index_to_delete)
frequencies = np.delete(frequencies, index_to_delete)

# Tworzenie wykresu w Plotly
fig = go.Figure()

# Dodawanie słupków na wykres
fig.add_trace(go.Bar(
    x=unique_values,
    y=frequencies,
    marker_color='#4287f5',  # Kolor słupków
    text=frequencies,
    textposition='outside',  # Wyświetlanie tekstu nad słupkiem
    textfont=dict(size=12, color='black')  # Konfiguracja czcionki tekstu
))

# Konfiguracja osi i tytułu wykresu
fig.update_layout(
    xaxis_title='Wiek biegaczek',
    yaxis_title='Liczba biegaczek, które ustanowiły rekord osobisty',
    title='Wiek vs Personal Best',
    xaxis=dict(type='linear'),  # Wymuszenie liniowego układu osi X
    yaxis=dict(showgrid=True, gridcolor='lightgray'),  # Wyświetlanie siatki na osi Y
    height=600
)

# Zapisywanie wykresu w pliku HTML
fig.write_html('personal_best_graph.html')

In [50]:

# Konwersja kolumny 'Wind' na typ float
df['Wind'] = pd.to_numeric(df['Wind'], errors='coerce')

# Usunięcie literek "A" na końcu niektórych wartości czasu
df['Time'] = df['Time'].str.rstrip('A')

# Usunięcie duplikatów
df = df.drop_duplicates(subset=['Time', 'Wind'])

# Podział danych na dodatni i ujemny wiatr
positive_wind = df[df['Wind'] >= 0]
negative_wind = df[df['Wind'] < 0]

# Tworzenie wykresu
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=positive_wind['Time'],
    y=positive_wind['Wind'],
    mode='markers',
    marker=dict(
        size=8,
        color='red'  # Zmiana koloru na czerwony
    ),
    name='Positive Wind'
))

fig.add_trace(go.Scatter(
    x=negative_wind['Time'],
    y=negative_wind['Wind'],
    mode='markers',
    marker=dict(
        size=8,
        color='blue'  # Zmiana koloru na niebieski
    ),
    name='Negative Wind'
))

# Ustawienia osi
fig.update_layout(
    title='Zależność między wiatrem a czasem',
    xaxis=dict(title='Czas (s)'),
    yaxis=dict(title='Wiatr', range=[-2.5, 2.5]),
    height= 600
)


# Wyświetlanie wykresu
fig.write_html('wind.html')